Fact table upsert

Set Catalog

In [0]:
spark.sql("USE CATALOG postnord")
print(spark.catalog.currentCatalog())

load delta table

In [0]:
df_status = spark.table("silver_item_status")

In [0]:
# Register df_status as a temp view
df_status.createOrReplaceTempView("updates")

# Create gold_parcel_delivery_quality table if it does not exist
spark.sql("""
CREATE TABLE IF NOT EXISTS gold_parcel_delivery_quality
USING DELTA
AS SELECT * FROM updates WHERE 1=0
""")



# Upsert into gold_parcel_delivery_quality
spark.sql("""
MERGE INTO gold_parcel_delivery_quality AS target
USING updates AS source
ON target.packageid = source.packageid AND target.systemdato = source.systemdato
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")

In [0]:
%sql
select * 
from gold_parcel_delivery_quality
limit 5;